In [1]:
import gspread
import time 
from datetime import datetime
import re
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
ciudades = ['Orlando, FL, EEUU'
'Miami, FL, EEUU'
'Tampa, FL, EEUU'
'Clearwater, FL, EEUU'
'Saint Petesburg, FL, EEUU'
'Brandon, FL, EEUU'
'Largo, FL, EEUU'
'Palm Harbor, FL, EEUU'
'Dunedin, FL, EEUU'
'Pinellas Park, FL, EEUU']
categoria = 'Mexican'

In [4]:
driver = webdriver.Chrome()
link = 'https://www.yelp.com/'
driver.get(link)

time.sleep(2)
category = driver.find_element(By.XPATH, '//*[@id="search_description"]')

category.clear()
category.click()
category.send_keys('Mexican')
time.sleep(1)

ciudad = driver.find_element(By.XPATH, '//*[@id="search_location"]')

ciudad.clear()
ciudad.click()
ciudad.send_keys(ciudades[0])
time.sleep(1)

buscar = driver.find_element(By.XPATH, '//*[@id="header_find_form"]/div[3]/button')

buscar.click()

restaurantes = driver.find_elements(By.CLASS_NAME, 'y-css-cxcdjj')
restaurante = restaurantes[0]

restaurante.click()
time.sleep(5)

ventana2 = driver.switch_to.window(driver.window_handles[1])
driver.close(ventana2)



AttributeError: 'NoneType' object has no attribute 'close'

In [31]:
restaurantes

[<selenium.webdriver.remote.webelement.WebElement (session="15d37e32e4f2eaa0b3ad9aff47cdb438", element="f.4DBAD7A9BF797EE6B078C4F99B7EE772.d.0675481759C1F004BA76E7859C4B22D8.e.148")>,
 <selenium.webdriver.remote.webelement.WebElement (session="15d37e32e4f2eaa0b3ad9aff47cdb438", element="f.4DBAD7A9BF797EE6B078C4F99B7EE772.d.0675481759C1F004BA76E7859C4B22D8.e.149")>,
 <selenium.webdriver.remote.webelement.WebElement (session="15d37e32e4f2eaa0b3ad9aff47cdb438", element="f.4DBAD7A9BF797EE6B078C4F99B7EE772.d.0675481759C1F004BA76E7859C4B22D8.e.150")>,
 <selenium.webdriver.remote.webelement.WebElement (session="15d37e32e4f2eaa0b3ad9aff47cdb438", element="f.4DBAD7A9BF797EE6B078C4F99B7EE772.d.0675481759C1F004BA76E7859C4B22D8.e.151")>,
 <selenium.webdriver.remote.webelement.WebElement (session="15d37e32e4f2eaa0b3ad9aff47cdb438", element="f.4DBAD7A9BF797EE6B078C4F99B7EE772.d.0675481759C1F004BA76E7859C4B22D8.e.152")>,
 <selenium.webdriver.remote.webelement.WebElement (session="15d37e32e4f2eaa0b3ad

In [38]:
restaurantes[3].click()

In [44]:
name = driver.find_element(By.XPATH, '/html/body/yelp-react-root/div[1]/div[4]/div[1]/div[1]/div/div/div[1]/h1').text

name

'U Pizza'